# 🧹 Maji Ndogo Data Ingestion & Cleaning Pipeline

This notebook performs structured ingestion and preprocessing of **Maji Ndogo’s agricultural field and weather data**. 

We leverage modular scripts to:
- Load field data from a local SQLite database
- Import weather data from remote CSV sources
- Standardize column names and correct known value anomalies
- Map field plots to weather stations

The result is a pair of cleaned and analysis-ready datasets that form the foundation for subsequent exploratory data analysis and validation.


# 1. Data Ingestion and Cleaning

This notebook handles the ingestion of field and weather data, their cleaning and integration into a single usable format.

We use the following scripts:
- `data_ingestion.py` – Handles DB connection and web-based CSV loading
- `field_data_processor.py` – Cleans and prepares field data
- `weather_data_processor.py` – Extracts and transforms raw weather logs


In [3]:
import re
import numpy as np
import pandas as pd
from field_data_processor import FieldDataProcessor
from weather_data_processor import WeatherDataProcessor
import logging 

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

config_params = {
    "sql_query": """
        SELECT *
        FROM geographic_features
        LEFT JOIN weather_features USING (Field_ID)
        LEFT JOIN soil_and_crop_features USING (Field_ID)
        LEFT JOIN farm_management_features USING (Field_ID)
    """,  # SQL query for extracting data
    "db_path": 'sqlite:///Maji_Ndogo_farm_survey_small.db',  # Path to the database
    "columns_to_rename": {
        'Annual_yield': 'Crop_type',
        'Crop_type': 'Annual_yield'
    },  # Dictionary of columns we want to swap the names of
    "values_to_rename": {
        'cassaval': 'cassava',
        'wheatn': 'wheat',
        'teaa': 'tea'
    },  # Crop type renaming dictionary
    "weather_csv_path": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv",  # Weather data CSV URL
    "weather_mapping_csv": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv",  # Weather data mapping CSV URL
    "regex_patterns": {
        'Rainfall': r'(\d+(\.\d+)?)\s?mm',
        'Temperature': r'(\d+(\.\d+)?)\s?C',
        'Pollution_level': r'=\s*(-?\d+(\.\d+)?)|Pollution at \s*(-?\d+(\.\d+)?)'
    }  # Regex patterns for processing messages
}


field_processor = FieldDataProcessor(config_params)
field_processor.process()
field_df = field_processor.df

weather_processor = WeatherDataProcessor(config_params)
weather_processor.process()
weather_df = weather_processor.weather_df

# Rename 'Ave_temps' in field_df to 'Temperature' to match weather_df
field_df.rename(columns={'Ave_temps': 'Temperature'}, inplace=True)

2025-06-12 18:03:42,541 - field_data_processor.FieldDataProcessor - INFO - Starting the data processing pipeline.
2025-06-12 18:03:42,541 - field_data_processor.FieldDataProcessor - INFO - Successfully loaded data.
2025-06-12 18:03:42,541 - data_ingestion - INFO - Database engine created successfully.
2025-06-12 18:03:42,764 - data_ingestion - INFO - Query executed successfully.
2025-06-12 18:03:42,764 - field_data_processor.FieldDataProcessor - INFO - Columns renamed successfully.
2025-06-12 18:03:42,775 - field_data_processor.FieldDataProcessor - INFO - Corrections applied successfully.
2025-06-12 18:03:43,493 - data_ingestion - INFO - CSV file read successfully from the web.
2025-06-12 18:03:43,539 - field_data_processor.FieldDataProcessor - INFO - Weather station mapping completed successfully.
2025-06-12 18:03:43,541 - field_data_processor.FieldDataProcessor - INFO - Data processing pipeline completed successfully.
2025-06-12 18:03:44,136 - data_ingestion - INFO - CSV file read su

In [4]:
field_df.head()

,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Temperature,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield,Unnamed: 0,Weather_station
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964,0,4
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302,1,0
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647,2,0
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348,3,1
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076,4,1


In [5]:
weather_df.head()

,Weather_station_ID,Message,Measurement,Value
0,0,【2022-01-04 21:47:48】温度感应: 现在温度是 12.82C.,Temperature,12.82
1,2,"环境监测报告: 2022-10-25 11:53:44, Air Quality Index...",Pollution_level,0.18
2,0,Temp. Reading [2023-05-23 09:41:36]: Current 1...,Temperature,14.53
3,0,"Weather Update: As of 2022-08-29 06:44:16, rai...",Rainfall,1917.49
4,4,Temp. Reading [2023-10-23 12:48:18]: Current 1...,Temperature,13.23


## 🧠 Summary

- Field and weather data successfully ingested.  
- Data cleaned, renamed, and joined via shared `Field_ID`.  
- Next step: perform Exploratory Data Analysis in the next notebook.
